In [4]:
import geopandas as gpd
import pandas as pd
from shapely import Polygon
import contextily as cx
from pathlib import Path
import shapelysmooth as shps
import shapely

All data files are assumed to be inside a `data` folder located at the project root.

In [5]:
cx_tile = cx.providers.CartoDB.Positron
data_dir = Path('.').absolute().parent / 'data'

In [6]:
xy_crs = "EPSG:6875"
gates_gdf = pd.read_parquet(data_dir / "VarchiAreaVerde.parquet")
gates_gdf = gpd.GeoDataFrame(
    gates_gdf,
    geometry=gpd.points_from_xy(
        gates_gdf["Longitudine"], gates_gdf["Latitudine"], crs="epsg:4326"
    ),
).to_crs(xy_crs)
p = Polygon(gates_gdf["geometry"])
green_area = gpd.GeoDataFrame(
    geometry=[shps.catmull_rom_smooth(shapely.concave_hull(p, ratio=0.4))],
    crs=gates_gdf.crs,
)

In [10]:
green_area.to_parquet(data_dir / 'green_area.parquet')

In [ ]:
m = green_area.explore(color="green", style_kwds={"fillOpacity": 0.2}, tiles=cx_tile)
gates_gdf.explore(m=m, marker_kwds={"radius": 10})
m